In [1]:
import numpy as np
import pandas as pd

import tensorflow
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical

In [2]:
''' defining some useful funcion '''

def loading_data(train=40000, test=10000, full=False):
    ''' reading data '''
    if full:
        s_df = pd.read_csv('sudoku.csv').values
    else:
        ''' converting s_df into array '''
        s_df = next(
            pd.read_csv('sudoku.csv', chunksize=(train + test))).values
        
    ''' transposing data '''
    q, sols = s_df.T
    X = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in q])
    y = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in sols])
    
    return (X[:train], y[:train]), (X[train:], y[train:])


def diff(grids_true, grids_pred):
    """ This function shows how well predicted quizzes fit to actual solutions. """
    return (grids_true != grids_pred).sum((1, 2))


def del_digits(X, delete=1):
    """ This function is used to create sudoku quizzes from solutions """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, delete)] = 0  # generate blanks (replace = True)
        
    return to_categorical(grids)


def batch_smart_solve(grids, solver):
    """ This function is ugly, feel free to optimize the code """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        pred = np.array(solver.predict(to_categorical(grids)))  # get predictions
        probs = pred.max(2).T  # get highest probability for each 81 digit to predict
        values = pred.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            '''  don't try to fill already completed grid'''
            if any(zero):  
                '''focus on blanks only'''
                where = np.where(zero)[0]  
                ''' best score FOR A ZERO VALUE (confident blank) '''
                conf_pos = where[prob[zero].argmax()] 
                ''' get corresponding value '''
                conf_val = value[conf_pos]  
                ''' fill digit inplace '''
                grid.flat[conf_pos] = conf_val  
    return grids

In [3]:
inp_shape = (9, 9, 10)
(_, y_train), (X_test, y_test) = loading_data() 

''' one-hot encoding'''
X_train = to_categorical(y_train).astype('float32')
X_test = to_categorical(y_test).astype('float32')

y_train = to_categorical(y_train-1).astype('float32') 
y_test = to_categorical(y_test-1).astype('float32')   

In [4]:
''' Model '''

''' container '''
model = Sequential()

''' 1st Hidden Layer '''
model.add(Dense(64, activation='relu', input_shape=inp_shape))
model.add(Dropout(0.4))

''' 2nd Hidden Layer '''
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))

''' flattening output '''
model.add(Flatten())

''' Input layer '''
inp = Input(shape=inp_shape)  
features = model(inp)  # commons features

''' Classification Layer '''
out = [Dense(9, activation='softmax')(features) for i in range(81)]

''' final model '''
model = Model(inp, out)  

''' compile the model '''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
''' training '''
model.fit(del_digits(X_train, 0), [y_train[:, i, j, :] for i in range(9) for j in range(9)], batch_size=128, epochs=1, 
           verbose=1)

158/313 [==============>...............] - ETA: 17s - loss: 82.5664 - dense_2_loss: 1.0277 - dense_3_loss: 1.0231 - dense_4_loss: 1.0327 - dense_5_loss: 1.0209 - dense_6_loss: 1.0134 - dense_7_loss: 1.0200 - dense_8_loss: 1.0187 - dense_9_loss: 1.0186 - dense_10_loss: 1.0248 - dense_11_loss: 1.0286 - dense_12_loss: 1.0298 - dense_13_loss: 1.0234 - dense_14_loss: 1.0189 - dense_15_loss: 1.0102 - dense_16_loss: 1.0239 - dense_17_loss: 1.0191 - dense_18_loss: 1.0323 - dense_19_loss: 1.0219 - dense_20_loss: 1.0057 - dense_21_loss: 1.0258 - dense_22_loss: 1.0302 - dense_23_loss: 1.0144 - dense_24_loss: 1.0353 - dense_25_loss: 1.0089 - dense_26_loss: 1.0080 - dense_27_loss: 1.0212 - dense_28_loss: 1.0167 - dense_29_loss: 1.0123 - dense_30_loss: 1.0165 - dense_31_loss: 1.0224 - dense_32_loss: 1.0261 - dense_33_loss: 1.0131 - dense_34_loss: 1.0168 - dense_35_loss: 1.0174 - dense_36_loss: 1.0209 - dense_37_loss: 1.0134 - dense_38_loss: 1.0142 - dense_39_loss: 1.0171 - dense_40_loss: 1.0178 - de

159/313 [==============>...............] - ETA: 17s - loss: 81.5922 - dense_2_loss: 1.0154 - dense_3_loss: 1.0111 - dense_4_loss: 1.0205 - dense_5_loss: 1.0089 - dense_6_loss: 1.0014 - dense_7_loss: 1.0080 - dense_8_loss: 1.0068 - dense_9_loss: 1.0066 - dense_10_loss: 1.0126 - dense_11_loss: 1.0165 - dense_12_loss: 1.0177 - dense_13_loss: 1.0114 - dense_14_loss: 1.0068 - dense_15_loss: 0.9983 - dense_16_loss: 1.0119 - dense_17_loss: 1.0073 - dense_18_loss: 1.0199 - dense_19_loss: 1.0099 - dense_20_loss: 0.9936 - dense_21_loss: 1.0137 - dense_22_loss: 1.0181 - dense_23_loss: 1.0023 - dense_24_loss: 1.0230 - dense_25_loss: 0.9970 - dense_26_loss: 0.9961 - dense_27_loss: 1.0092 - dense_28_loss: 1.0046 - dense_29_loss: 1.0003 - dense_30_loss: 1.0045 - dense_31_loss: 1.0102 - dense_32_loss: 1.0142 - dense_33_loss: 1.0012 - dense_34_loss: 1.0047 - dense_35_loss: 1.0054 - dense_36_loss: 1.0089 - dense_37_loss: 1.0015 - dense_38_loss: 1.0021 - dense_39_loss: 1.0050 - dense_40_loss: 1.0057 - de

160/313 [==============>...............] - ETA: 17s - loss: 81.1135 - dense_2_loss: 1.0094 - dense_3_loss: 1.0051 - dense_4_loss: 1.0144 - dense_5_loss: 1.0030 - dense_6_loss: 0.9955 - dense_7_loss: 1.0020 - dense_8_loss: 1.0009 - dense_9_loss: 1.0006 - dense_10_loss: 1.0067 - dense_11_loss: 1.0108 - dense_12_loss: 1.0118 - dense_13_loss: 1.0056 - dense_14_loss: 1.0009 - dense_15_loss: 0.9924 - dense_16_loss: 1.0060 - dense_17_loss: 1.0013 - dense_18_loss: 1.0139 - dense_19_loss: 1.0041 - dense_20_loss: 0.9877 - dense_21_loss: 1.0076 - dense_22_loss: 1.0122 - dense_23_loss: 0.9963 - dense_24_loss: 1.0170 - dense_25_loss: 0.9910 - dense_26_loss: 0.9902 - dense_27_loss: 1.0034 - dense_28_loss: 0.9988 - dense_29_loss: 0.9944 - dense_30_loss: 0.9988 - dense_31_loss: 1.0042 - dense_32_loss: 1.0082 - dense_33_loss: 0.9953 - dense_34_loss: 0.9988 - dense_35_loss: 0.9995 - dense_36_loss: 1.0030 - dense_37_loss: 0.9957 - dense_38_loss: 0.9962 - dense_39_loss: 0.9991 - dense_40_loss: 0.9999 - de

161/313 [==============>...............] - ETA: 16s - loss: 80.6435 - dense_2_loss: 1.0035 - dense_3_loss: 0.9993 - dense_4_loss: 1.0089 - dense_5_loss: 0.9972 - dense_6_loss: 0.9899 - dense_7_loss: 0.9963 - dense_8_loss: 0.9950 - dense_9_loss: 0.9947 - dense_10_loss: 1.0010 - dense_11_loss: 1.0050 - dense_12_loss: 1.0060 - dense_13_loss: 0.9997 - dense_14_loss: 0.9950 - dense_15_loss: 0.9867 - dense_16_loss: 1.0002 - dense_17_loss: 0.9955 - dense_18_loss: 1.0079 - dense_19_loss: 0.9981 - dense_20_loss: 0.9819 - dense_21_loss: 1.0017 - dense_22_loss: 1.0063 - dense_23_loss: 0.9905 - dense_24_loss: 1.0112 - dense_25_loss: 0.9851 - dense_26_loss: 0.9843 - dense_27_loss: 0.9978 - dense_28_loss: 0.9930 - dense_29_loss: 0.9885 - dense_30_loss: 0.9930 - dense_31_loss: 0.9984 - dense_32_loss: 1.0023 - dense_33_loss: 0.9894 - dense_34_loss: 0.9930 - dense_35_loss: 0.9937 - dense_36_loss: 0.9972 - dense_37_loss: 0.9900 - dense_38_loss: 0.9904 - dense_39_loss: 0.9932 - dense_40_loss: 0.9942 - de

162/313 [==============>...............] - ETA: 16s - loss: 80.1759 - dense_2_loss: 0.9977 - dense_3_loss: 0.9934 - dense_4_loss: 1.0030 - dense_5_loss: 0.9915 - dense_6_loss: 0.9843 - dense_7_loss: 0.9905 - dense_8_loss: 0.9893 - dense_9_loss: 0.9891 - dense_10_loss: 0.9953 - dense_11_loss: 0.9991 - dense_12_loss: 1.0001 - dense_13_loss: 0.9939 - dense_14_loss: 0.9892 - dense_15_loss: 0.9810 - dense_16_loss: 0.9942 - dense_17_loss: 0.9897 - dense_18_loss: 1.0020 - dense_19_loss: 0.9922 - dense_20_loss: 0.9762 - dense_21_loss: 0.9960 - dense_22_loss: 1.0003 - dense_23_loss: 0.9850 - dense_24_loss: 1.0052 - dense_25_loss: 0.9793 - dense_26_loss: 0.9787 - dense_27_loss: 0.9922 - dense_28_loss: 0.9875 - dense_29_loss: 0.9827 - dense_30_loss: 0.9872 - dense_31_loss: 0.9926 - dense_32_loss: 0.9965 - dense_33_loss: 0.9837 - dense_34_loss: 0.9873 - dense_35_loss: 0.9880 - dense_36_loss: 0.9915 - dense_37_loss: 0.9842 - dense_38_loss: 0.9846 - dense_39_loss: 0.9875 - dense_40_loss: 0.9886 - de

...

173/313 [===============>..............] - ETA: 15s - loss: 75.3676 - dense_2_loss: 0.9377 - dense_3_loss: 0.9338 - dense_4_loss: 0.9429 - dense_5_loss: 0.9330 - dense_6_loss: 0.9251 - dense_7_loss: 0.9308 - dense_8_loss: 0.9297 - dense_9_loss: 0.9298 - dense_10_loss: 0.9358 - dense_11_loss: 0.9395 - dense_12_loss: 0.9404 - dense_13_loss: 0.9341 - dense_14_loss: 0.9301 - dense_15_loss: 0.9227 - dense_16_loss: 0.9347 - dense_17_loss: 0.9303 - dense_18_loss: 0.9419 - dense_19_loss: 0.9329 - dense_20_loss: 0.9176 - dense_21_loss: 0.9362 - dense_22_loss: 0.9401 - dense_23_loss: 0.9261 - dense_24_loss: 0.9450 - dense_25_loss: 0.9208 - dense_26_loss: 0.9198 - dense_27_loss: 0.9327 - dense_28_loss: 0.9282 - dense_29_loss: 0.9241 - dense_30_loss: 0.9282 - dense_31_loss: 0.9334 - dense_32_loss: 0.9369 - dense_33_loss: 0.9240 - dense_34_loss: 0.9277 - dense_35_loss: 0.9286 - dense_36_loss: 0.9322 - dense_37_loss: 0.9255 - dense_38_loss: 0.9257 - dense_39_loss: 0.9278 - dense_40_loss: 0.9289 - de

174/313 [===============>..............] - ETA: 15s - loss: 74.9589 - dense_2_loss: 0.9326 - dense_3_loss: 0.9288 - dense_4_loss: 0.9378 - dense_5_loss: 0.9280 - dense_6_loss: 0.9201 - dense_7_loss: 0.9257 - dense_8_loss: 0.9247 - dense_9_loss: 0.9247 - dense_10_loss: 0.9308 - dense_11_loss: 0.9345 - dense_12_loss: 0.9354 - dense_13_loss: 0.9291 - dense_14_loss: 0.9250 - dense_15_loss: 0.9178 - dense_16_loss: 0.9296 - dense_17_loss: 0.9252 - dense_18_loss: 0.9368 - dense_19_loss: 0.9277 - dense_20_loss: 0.9125 - dense_21_loss: 0.9311 - dense_22_loss: 0.9350 - dense_23_loss: 0.9211 - dense_24_loss: 0.9397 - dense_25_loss: 0.9157 - dense_26_loss: 0.9148 - dense_27_loss: 0.9277 - dense_28_loss: 0.9232 - dense_29_loss: 0.9190 - dense_30_loss: 0.9232 - dense_31_loss: 0.9284 - dense_32_loss: 0.9318 - dense_33_loss: 0.9190 - dense_34_loss: 0.9227 - dense_35_loss: 0.9235 - dense_36_loss: 0.9272 - dense_37_loss: 0.9205 - dense_38_loss: 0.9205 - dense_39_loss: 0.9227 - dense_40_loss: 0.9238 - de

175/313 [===============>..............] - ETA: 15s - loss: 74.5542 - dense_2_loss: 0.9275 - dense_3_loss: 0.9238 - dense_4_loss: 0.9327 - dense_5_loss: 0.9230 - dense_6_loss: 0.9152 - dense_7_loss: 0.9208 - dense_8_loss: 0.9197 - dense_9_loss: 0.9196 - dense_10_loss: 0.9258 - dense_11_loss: 0.9294 - dense_12_loss: 0.9304 - dense_13_loss: 0.9241 - dense_14_loss: 0.9200 - dense_15_loss: 0.9129 - dense_16_loss: 0.9246 - dense_17_loss: 0.9202 - dense_18_loss: 0.9317 - dense_19_loss: 0.9228 - dense_20_loss: 0.9075 - dense_21_loss: 0.9262 - dense_22_loss: 0.9299 - dense_23_loss: 0.9160 - dense_24_loss: 0.9347 - dense_25_loss: 0.9107 - dense_26_loss: 0.9099 - dense_27_loss: 0.9226 - dense_28_loss: 0.9182 - dense_29_loss: 0.9140 - dense_30_loss: 0.9182 - dense_31_loss: 0.9234 - dense_32_loss: 0.9267 - dense_33_loss: 0.9140 - dense_34_loss: 0.9177 - dense_35_loss: 0.9186 - dense_36_loss: 0.9221 - dense_37_loss: 0.9154 - dense_38_loss: 0.9156 - dense_39_loss: 0.9178 - dense_40_loss: 0.9188 - de

176/313 [===============>..............] - ETA: 15s - loss: 74.1536 - dense_2_loss: 0.9225 - dense_3_loss: 0.9189 - dense_4_loss: 0.9276 - dense_5_loss: 0.9180 - dense_6_loss: 0.9102 - dense_7_loss: 0.9158 - dense_8_loss: 0.9149 - dense_9_loss: 0.9147 - dense_10_loss: 0.9208 - dense_11_loss: 0.9243 - dense_12_loss: 0.9253 - dense_13_loss: 0.9193 - dense_14_loss: 0.9150 - dense_15_loss: 0.9080 - dense_16_loss: 0.9197 - dense_17_loss: 0.9154 - dense_18_loss: 0.9267 - dense_19_loss: 0.9179 - dense_20_loss: 0.9027 - dense_21_loss: 0.9211 - dense_22_loss: 0.9248 - dense_23_loss: 0.9111 - dense_24_loss: 0.9297 - dense_25_loss: 0.9057 - dense_26_loss: 0.9051 - dense_27_loss: 0.9176 - dense_28_loss: 0.9133 - dense_29_loss: 0.9090 - dense_30_loss: 0.9134 - dense_31_loss: 0.9184 - dense_32_loss: 0.9219 - dense_33_loss: 0.9091 - dense_34_loss: 0.9127 - dense_35_loss: 0.9136 - dense_36_loss: 0.9171 - dense_37_loss: 0.9103 - dense_38_loss: 0.9106 - dense_39_loss: 0.9128 - dense_40_loss: 0.9138 - de

313/313 [==============================] - 35s 113ms/step - loss: 42.6864 - dense_2_loss: 0.5322 - dense_3_loss: 0.5285 - dense_4_loss: 0.5339 - dense_5_loss: 0.5281 - dense_6_loss: 0.5239 - dense_7_loss: 0.5273 - dense_8_loss: 0.5262 - dense_9_loss: 0.5264 - dense_10_loss: 0.5296 - dense_11_loss: 0.5328 - dense_12_loss: 0.5322 - dense_13_loss: 0.5296 - dense_14_loss: 0.5263 - dense_15_loss: 0.5231 - dense_16_loss: 0.5290 - dense_17_loss: 0.5275 - dense_18_loss: 0.5336 - dense_19_loss: 0.5289 - dense_20_loss: 0.5197 - dense_21_loss: 0.5304 - dense_22_loss: 0.5327 - dense_23_loss: 0.5253 - dense_24_loss: 0.5353 - dense_25_loss: 0.5215 - dense_26_loss: 0.5207 - dense_27_loss: 0.5277 - dense_28_loss: 0.5262 - dense_29_loss: 0.5238 - dense_30_loss: 0.5249 - dense_31_loss: 0.5287 - dense_32_loss: 0.5305 - dense_33_loss: 0.5236 - dense_34_loss: 0.5253 - dense_35_loss: 0.5263 - dense_36_loss: 0.5285 - dense_37_loss: 0.5248 - dense_38_loss: 0.5243 - dense_39_loss: 0.5256 - dense_40_loss: 0.525

In [ ]:
''' Early Stopping '''
es = EarlyStopping(patience=2, verbose=1)

i = 1
for ep, dele in zip([1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]):

    print('Pass n° {} ...'.format(i))
    i += 1
    
    ''' training '''
    model.fit(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)],
              validation_data=(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)]), 
              batch_size=128, epochs=ep, verbose=1, callbacks=[es])

Pass n° 1 ...
313/313 [==============================] - 57s 184ms/step - loss: 1.2739 - dense_2_loss: 0.0161 - dense_3_loss: 0.0155 - dense_4_loss: 0.0161 - dense_5_loss: 0.0151 - dense_6_loss: 0.0157 - dense_7_loss: 0.0171 - dense_8_loss: 0.0157 - dense_9_loss: 0.0165 - dense_10_loss: 0.0157 - dense_11_loss: 0.0167 - dense_12_loss: 0.0148 - dense_13_loss: 0.0155 - dense_14_loss: 0.0180 - dense_15_loss: 0.0144 - dense_16_loss: 0.0161 - dense_17_loss: 0.0147 - dense_18_loss: 0.0157 - dense_19_loss: 0.0154 - dense_20_loss: 0.0153 - dense_21_loss: 0.0162 - dense_22_loss: 0.0152 - dense_23_loss: 0.0163 - dense_24_loss: 0.0153 - dense_25_loss: 0.0153 - dense_26_loss: 0.0161 - dense_27_loss: 0.0175 - dense_28_loss: 0.0159 - dense_29_loss: 0.0177 - dense_30_loss: 0.0173 - dense_31_loss: 0.0178 - dense_32_loss: 0.0144 - dense_33_loss: 0.0148 - dense_34_loss: 0.0162 - dense_35_loss: 0.0153 - dense_36_loss: 0.0151 - dense_37_loss: 0.0154 - dense_38_loss: 0.0153 - dense_39_loss: 0.0143 - dense_4

Pass n° 2 ...
Epoch 1/2
313/313 [==============================] - 53s 169ms/step - loss: 1.4523 - dense_2_loss: 0.0162 - dense_3_loss: 0.0169 - dense_4_loss: 0.0191 - dense_5_loss: 0.0175 - dense_6_loss: 0.0209 - dense_7_loss: 0.0189 - dense_8_loss: 0.0168 - dense_9_loss: 0.0164 - dense_10_loss: 0.0190 - dense_11_loss: 0.0173 - dense_12_loss: 0.0194 - dense_13_loss: 0.0171 - dense_14_loss: 0.0168 - dense_15_loss: 0.0172 - dense_16_loss: 0.0190 - dense_17_loss: 0.0169 - dense_18_loss: 0.0150 - dense_19_loss: 0.0153 - dense_20_loss: 0.0173 - dense_21_loss: 0.0190 - dense_22_loss: 0.0179 - dense_23_loss: 0.0192 - dense_24_loss: 0.0188 - dense_25_loss: 0.0163 - dense_26_loss: 0.0185 - dense_27_loss: 0.0178 - dense_28_loss: 0.0178 - dense_29_loss: 0.0178 - dense_30_loss: 0.0180 - dense_31_loss: 0.0173 - dense_32_loss: 0.0181 - dense_33_loss: 0.0193 - dense_34_loss: 0.0183 - dense_35_loss: 0.0184 - dense_36_loss: 0.0171 - dense_37_loss: 0.0168 - dense_38_loss: 0.0164 - dense_39_loss: 0.0178

Epoch 2/2
313/313 [==============================] - 56s 178ms/step - loss: 1.0499 - dense_2_loss: 0.0114 - dense_3_loss: 0.0140 - dense_4_loss: 0.0142 - dense_5_loss: 0.0141 - dense_6_loss: 0.0129 - dense_7_loss: 0.0145 - dense_8_loss: 0.0127 - dense_9_loss: 0.0133 - dense_10_loss: 0.0124 - dense_11_loss: 0.0117 - dense_12_loss: 0.0120 - dense_13_loss: 0.0130 - dense_14_loss: 0.0116 - dense_15_loss: 0.0134 - dense_16_loss: 0.0136 - dense_17_loss: 0.0135 - dense_18_loss: 0.0147 - dense_19_loss: 0.0167 - dense_20_loss: 0.0167 - dense_21_loss: 0.0101 - dense_22_loss: 0.0135 - dense_23_loss: 0.0120 - dense_24_loss: 0.0126 - dense_25_loss: 0.0128 - dense_26_loss: 0.0120 - dense_27_loss: 0.0134 - dense_28_loss: 0.0126 - dense_29_loss: 0.0115 - dense_30_loss: 0.0112 - dense_31_loss: 0.0132 - dense_32_loss: 0.0141 - dense_33_loss: 0.0119 - dense_34_loss: 0.0129 - dense_35_loss: 0.0125 - dense_36_loss: 0.0132 - dense_37_loss: 0.0131 - dense_38_loss: 0.0146 - dense_39_loss: 0.0125 - dense_40_lo

Pass n° 3 ...
Epoch 1/3
313/313 [==============================] - 56s 178ms/step - loss: 1.7041 - dense_2_loss: 0.0231 - dense_3_loss: 0.0212 - dense_4_loss: 0.0238 - dense_5_loss: 0.0195 - dense_6_loss: 0.0198 - dense_7_loss: 0.0197 - dense_8_loss: 0.0194 - dense_9_loss: 0.0226 - dense_10_loss: 0.0183 - dense_11_loss: 0.0232 - dense_12_loss: 0.0226 - dense_13_loss: 0.0203 - dense_14_loss: 0.0218 - dense_15_loss: 0.0215 - dense_16_loss: 0.0208 - dense_17_loss: 0.0205 - dense_18_loss: 0.0191 - dense_19_loss: 0.0204 - dense_20_loss: 0.0208 - dense_21_loss: 0.0200 - dense_22_loss: 0.0188 - dense_23_loss: 0.0214 - dense_24_loss: 0.0221 - dense_25_loss: 0.0215 - dense_26_loss: 0.0219 - dense_27_loss: 0.0188 - dense_28_loss: 0.0214 - dense_29_loss: 0.0221 - dense_30_loss: 0.0231 - dense_31_loss: 0.0212 - dense_32_loss: 0.0232 - dense_33_loss: 0.0221 - dense_34_loss: 0.0201 - dense_35_loss: 0.0198 - dense_36_loss: 0.0202 - dense_37_loss: 0.0235 - dense_38_loss: 0.0204 - dense_39_loss: 0.0228

Epoch 2/3
313/313 [==============================] - 58s 187ms/step - loss: 1.0768 - dense_2_loss: 0.0150 - dense_3_loss: 0.0129 - dense_4_loss: 0.0132 - dense_5_loss: 0.0139 - dense_6_loss: 0.0130 - dense_7_loss: 0.0132 - dense_8_loss: 0.0140 - dense_9_loss: 0.0135 - dense_10_loss: 0.0148 - dense_11_loss: 0.0135 - dense_12_loss: 0.0144 - dense_13_loss: 0.0132 - dense_14_loss: 0.0124 - dense_15_loss: 0.0124 - dense_16_loss: 0.0140 - dense_17_loss: 0.0139 - dense_18_loss: 0.0143 - dense_19_loss: 0.0121 - dense_20_loss: 0.0133 - dense_21_loss: 0.0129 - dense_22_loss: 0.0136 - dense_23_loss: 0.0120 - dense_24_loss: 0.0123 - dense_25_loss: 0.0154 - dense_26_loss: 0.0140 - dense_27_loss: 0.0126 - dense_28_loss: 0.0132 - dense_29_loss: 0.0131 - dense_30_loss: 0.0127 - dense_31_loss: 0.0117 - dense_32_loss: 0.0120 - dense_33_loss: 0.0128 - dense_34_loss: 0.0133 - dense_35_loss: 0.0119 - dense_36_loss: 0.0132 - dense_37_loss: 0.0122 - dense_38_loss: 0.0133 - dense_39_loss: 0.0148 - dense_40_lo

...

Pass n° 5 ...
Epoch 1/6
313/313 [==============================] - 56s 179ms/step - loss: 1.7577 - dense_2_loss: 0.0215 - dense_3_loss: 0.0229 - dense_4_loss: 0.0234 - dense_5_loss: 0.0224 - dense_6_loss: 0.0208 - dense_7_loss: 0.0221 - dense_8_loss: 0.0222 - dense_9_loss: 0.0240 - dense_10_loss: 0.0223 - dense_11_loss: 0.0262 - dense_12_loss: 0.0218 - dense_13_loss: 0.0241 - dense_14_loss: 0.0215 - dense_15_loss: 0.0250 - dense_16_loss: 0.0206 - dense_17_loss: 0.0215 - dense_18_loss: 0.0212 - dense_19_loss: 0.0210 - dense_20_loss: 0.0200 - dense_21_loss: 0.0210 - dense_22_loss: 0.0204 - dense_23_loss: 0.0204 - dense_24_loss: 0.0201 - dense_25_loss: 0.0225 - dense_26_loss: 0.0233 - dense_27_loss: 0.0196 - dense_28_loss: 0.0202 - dense_29_loss: 0.0215 - dense_30_loss: 0.0220 - dense_31_loss: 0.0211 - dense_32_loss: 0.0205 - dense_33_loss: 0.0218 - dense_34_loss: 0.0211 - dense_35_loss: 0.0221 - dense_36_loss: 0.0221 - dense_37_loss: 0.0197 - dense_38_loss: 0.0219 - dense_39_loss: 0.0216

Epoch 2/6
313/313 [==============================] - 55s 176ms/step - loss: 1.0548 - dense_2_loss: 0.0131 - dense_3_loss: 0.0129 - dense_4_loss: 0.0137 - dense_5_loss: 0.0130 - dense_6_loss: 0.0131 - dense_7_loss: 0.0119 - dense_8_loss: 0.0133 - dense_9_loss: 0.0132 - dense_10_loss: 0.0137 - dense_11_loss: 0.0122 - dense_12_loss: 0.0137 - dense_13_loss: 0.0125 - dense_14_loss: 0.0121 - dense_15_loss: 0.0129 - dense_16_loss: 0.0131 - dense_17_loss: 0.0136 - dense_18_loss: 0.0129 - dense_19_loss: 0.0133 - dense_20_loss: 0.0122 - dense_21_loss: 0.0128 - dense_22_loss: 0.0131 - dense_23_loss: 0.0123 - dense_24_loss: 0.0143 - dense_25_loss: 0.0128 - dense_26_loss: 0.0140 - dense_27_loss: 0.0126 - dense_28_loss: 0.0141 - dense_29_loss: 0.0141 - dense_30_loss: 0.0116 - dense_31_loss: 0.0136 - dense_32_loss: 0.0117 - dense_33_loss: 0.0110 - dense_34_loss: 0.0141 - dense_35_loss: 0.0132 - dense_36_loss: 0.0121 - dense_37_loss: 0.0119 - dense_38_loss: 0.0131 - dense_39_loss: 0.0138 - dense_40_lo

Epoch 3/6
313/313 [==============================] - 58s 185ms/step - loss: 0.8737 - dense_2_loss: 0.0093 - dense_3_loss: 0.0131 - dense_4_loss: 0.0110 - dense_5_loss: 0.0141 - dense_6_loss: 0.0117 - dense_7_loss: 0.0098 - dense_8_loss: 0.0108 - dense_9_loss: 0.0097 - dense_10_loss: 0.0099 - dense_11_loss: 0.0105 - dense_12_loss: 0.0119 - dense_13_loss: 0.0099 - dense_14_loss: 0.0098 - dense_15_loss: 0.0108 - dense_16_loss: 0.0107 - dense_17_loss: 0.0098 - dense_18_loss: 0.0101 - dense_19_loss: 0.0106 - dense_20_loss: 0.0109 - dense_21_loss: 0.0103 - dense_22_loss: 0.0099 - dense_23_loss: 0.0099 - dense_24_loss: 0.0106 - dense_25_loss: 0.0095 - dense_26_loss: 0.0107 - dense_27_loss: 0.0096 - dense_28_loss: 0.0116 - dense_29_loss: 0.0120 - dense_30_loss: 0.0107 - dense_31_loss: 0.0110 - dense_32_loss: 0.0109 - dense_33_loss: 0.0112 - dense_34_loss: 0.0108 - dense_35_loss: 0.0123 - dense_36_loss: 0.0101 - dense_37_loss: 0.0113 - dense_38_loss: 0.0125 - dense_39_loss: 0.0130 - dense_40_lo

Epoch 4/6
313/313 [==============================] - 58s 184ms/step - loss: 0.7734 - dense_2_loss: 0.0104 - dense_3_loss: 0.0087 - dense_4_loss: 0.0123 - dense_5_loss: 0.0104 - dense_6_loss: 0.0097 - dense_7_loss: 0.0090 - dense_8_loss: 0.0096 - dense_9_loss: 0.0091 - dense_10_loss: 0.0103 - dense_11_loss: 0.0078 - dense_12_loss: 0.0095 - dense_13_loss: 0.0083 - dense_14_loss: 0.0094 - dense_15_loss: 0.0093 - dense_16_loss: 0.0083 - dense_17_loss: 0.0098 - dense_18_loss: 0.0093 - dense_19_loss: 0.0091 - dense_20_loss: 0.0097 - dense_21_loss: 0.0101 - dense_22_loss: 0.0099 - dense_23_loss: 0.0088 - dense_24_loss: 0.0090 - dense_25_loss: 0.0076 - dense_26_loss: 0.0088 - dense_27_loss: 0.0088 - dense_28_loss: 0.0098 - dense_29_loss: 0.0103 - dense_30_loss: 0.0105 - dense_31_loss: 0.0096 - dense_32_loss: 0.0092 - dense_33_loss: 0.0097 - dense_34_loss: 0.0084 - dense_35_loss: 0.0122 - dense_36_loss: 0.0095 - dense_37_loss: 0.0092 - dense_38_loss: 0.0099 - dense_39_loss: 0.0098 - dense_40_lo

Epoch 5/6
313/313 [==============================] - 54s 173ms/step - loss: 0.7232 - dense_2_loss: 0.0103 - dense_3_loss: 0.0090 - dense_4_loss: 0.0097 - dense_5_loss: 0.0078 - dense_6_loss: 0.0085 - dense_7_loss: 0.0105 - dense_8_loss: 0.0083 - dense_9_loss: 0.0092 - dense_10_loss: 0.0091 - dense_11_loss: 0.0083 - dense_12_loss: 0.0076 - dense_13_loss: 0.0087 - dense_14_loss: 0.0094 - dense_15_loss: 0.0083 - dense_16_loss: 0.0095 - dense_17_loss: 0.0098 - dense_18_loss: 0.0086 - dense_19_loss: 0.0092 - dense_20_loss: 0.0088 - dense_21_loss: 0.0086 - dense_22_loss: 0.0091 - dense_23_loss: 0.0109 - dense_24_loss: 0.0093 - dense_25_loss: 0.0093 - dense_26_loss: 0.0095 - dense_27_loss: 0.0096 - dense_28_loss: 0.0098 - dense_29_loss: 0.0079 - dense_30_loss: 0.0090 - dense_31_loss: 0.0084 - dense_32_loss: 0.0090 - dense_33_loss: 0.0100 - dense_34_loss: 0.0092 - dense_35_loss: 0.0093 - dense_36_loss: 0.0076 - dense_37_loss: 0.0094 - dense_38_loss: 0.0082 - dense_39_loss: 0.0093 - dense_40_lo

In [ ]:
'''Evaluating model '''
q = X_test.argmax(3)  
actual = y_test.argmax(3) + 1 

''' make some guesses'''
sg = batch_smart_solve(q, model)   

''' get number of errors on each quizz '''
de = diff(actual, sg)  

'''portion of correct solved quizzes '''
acc = (de == 0).mean()  

In [ ]:
print("""Grid solved:\t {} Correct ones:\t {} Accuracy:\t {}""".format(de.shape[0], (de==0).sum(), acc))